# Pandas alternatives Proof of Concept
In this notebook we will explore, if it's worth using pandas alternatives - vaex, spark, dask or modin if our dataset fits comfortably into the memory. We will load two files, join them, group by and aggregate and sort the results. The file `Performance_test.py` contains all these steps executable as a performance test. 

In [1]:
import os
import sys
import gc
from time import time, sleep, strftime, localtime

import pandas as pd
import dask.dataframe as dd
import modin.pandas as mpd
import vaex
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg
# pandas on ray has moved to Modin
# import ray.dataframe as rpd

In [2]:
# data based on https://www.kaggle.com/c/ieee-fraud-detection/data
folder = "/Data/Kaggle/EEE-CIS_Fraud_Detection"
files = ["train_transaction.csv", "train_identity.csv"]
paths = [os.path.join(folder, f) for f in files]

In [3]:
stats = {}

# Pandas

In [4]:
pd.__version__

'1.1.4'

In [5]:
stats["pandas"] = {}
s = stats["pandas"]

ts = time()
df = pd.read_csv(paths[0])
te = time()
s["load_transactions"] = te-ts

ts = time()
df2 = pd.read_csv(paths[1])
te = time()
s["load_identity"] = te-ts

ts = time()
dff = df.merge(df2, on="TransactionID")
te = time()
s["merge"] = te-ts

ts = time()
grp = dff.groupby(["isFraud","ProductCD","card4","card6","id_15","id_31"])["TransactionAmt"].agg(["mean","sum"])
te = time()
s["aggregation"] = te-ts

ts = time()
dff.sort_values(by=["card1","addr1","D9"], inplace=True)
dff.sort_values(by=["addr1","D9","card1"], inplace=True)
dff.sort_values(by=["D9","card1","addr1"], inplace=True)
te = time()
s["sorting"] = te-ts

ts = time()
dff["card2"].value_counts()
te = time()
s["value_counts"] = te-ts

In [6]:
pd.DataFrame(stats)

,pandas
aggregation,0.062498
load_identity,0.501993
load_transactions,20.143912
merge,2.207136
sorting,1.258234
value_counts,0.002657


In [7]:
dff.to_pickle("data/dff.pkl")

In [8]:
# Because julia groups by including N\A, let's just check that number of groups matches
grp = dff[["isFraud","ProductCD","card4","card6","id_15","id_31","TransactionAmt"]].fillna("~U~")\
.groupby(["isFraud","ProductCD","card4","card6","id_15","id_31"])["TransactionAmt"].agg(["mean","sum"])
grp.shape

(4553, 2)

In [9]:
def clean(wait_time: int=15):
    """Cleans created DataFrames and call the garbage collector to actions. Wait for 15s by default"""
    df, df2, dff, grp = None, None, None, None
    gc.collect()
    sleep(wait_time)
    return None

In [10]:
clean()


In [11]:
def list_variables_memory_usage() -> dict:
    """Memory of existing local variables"""
    local_vars = list(locals().items())
    return {var: sys.getsizeof(obj) for var, obj in local_vars}

# Dask
When to use dask - https://docs.dask.org/en/latest/dataframe.html#common-uses-and-anti-uses

In [12]:
stats["dask"] = {}
s = stats["dask"]

ts = time()
df = dd.read_csv(paths[0])
te = time()
s["load_transactions"] = te-ts

ts = time()
df2 = dd.read_csv(paths[1])
te = time()
s["load_identity"] = te-ts

ts = time()
dff = df.merge(df2, on="TransactionID")
te = time()
s["merge"] = te-ts

# the difference is that we call compute method, which runs all the computations at this point
ts = time()
grp = dff.groupby(["isFraud","ProductCD","card4","card6","id_15","id_31"])["TransactionAmt"]\
    .agg(["mean","sum"])\
    .compute()
te = time()
s["aggregation"] = te-ts

# parallel soring is tricky that is why there are only work arounds in dask. 
ts = time()
dfs = dff.set_index("card1").compute()
te = time()
s["sorting"] = te-ts


Results of the `.compute()` operation can be pandas dataframe rather than dask.dataframe.

In [13]:
type(dff), type(grp), type(dfs)

(dask.dataframe.core.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame)

In [14]:
clean()

In [15]:
pd.DataFrame(stats)

,pandas,dask
load_transactions,20.143912,0.090847
load_identity,0.501993,0.017655
merge,2.207136,0.075063
aggregation,0.062498,16.052712
sorting,1.258234,43.264110
value_counts,0.002657,NaN


In [16]:
stats["dask_indexed"] = {}
s = stats["dask_indexed"]

ts = time()
df = dd.read_csv(paths[0]).set_index("TransactionID")
te = time()
s["load_transactions"] = te-ts

ts = time()
df2 = dd.read_csv(paths[1]).set_index("TransactionID")
te = time()
s["load_identity"] = te-ts

ts = time()
dff = df.merge(df2, left_index=True, right_index=True)
te = time()
s["merge"] = te-ts

# the difference is that we call compute method, which runs all the computations at this point
ts = time()
grp = dff.groupby(["isFraud","ProductCD","card4","card6","id_15","id_31"])["TransactionAmt"]\
    .agg(["mean","sum"])\
    .compute()
te = time()
s["aggregation"] = te-ts

# parallel soring is tricky that is why there are only work arounds in dask. 
ts = time()
dff.set_index("card1").compute()
te = time()
s["sorting"] = te-ts


In [17]:
clean()
pd.DataFrame(stats)

,pandas,dask,dask_indexed
load_transactions,20.143912,0.090847,10.722161
load_identity,0.501993,0.017655,0.582975
merge,2.207136,0.075063,0.048098
aggregation,0.062498,16.052712,14.059959
sorting,1.258234,43.264110,40.348259
value_counts,0.002657,NaN,NaN


# Vaex

In [18]:
vaex.__version__

{'vaex-core': '2.0.3',
 'vaex-viz': '0.4.0',
 'vaex-hdf5': '0.6.0',
 'vaex-server': '0.3.1',
 'vaex-astro': '0.7.0',
 'vaex-jupyter': '0.5.2',
 'vaex-ml': '0.9.0',
 'vaex-arrow': '0.5.1'}

In [19]:
tool = "vaex"
stats[tool] = {}
s = stats[tool]


ts = time()
#df = vaex.open(paths[0])
df = vaex.from_csv(paths[0])
te = time()
s["load_transactions"] = te-ts

ts = time()
df2 = vaex.open(paths[1])
te = time()
s["load_identity"] = te-ts


In [20]:
ts = time()
dff = df.join(df2, on="TransactionID")
te = time()
s["merge"] = te-ts

In [21]:
# the difference is that we call compute method, which runs all the computations at this point
ts = time()

grp = dff.groupby(["isFraud","ProductCD","card4","card6","id_15","id_31"], 
                  agg={'TransactionAmt':["mean","sum"]})

# alternatively
#grp = dff.groupby([dff["isFraud"],dff["ProductCD"],dff["card4"],dff["card6"],dff["id_15"],dff["id_31"]], 
#                  agg=[vaex.agg.mean('TransactionAmt'), vaex.agg.sum('TransactionAmt')])


te = time()
s["aggregation"] = te-ts


In [22]:
# the difference is that we call compute method, which runs all the computations at this point
ts = time()
dff_s = dff.sort(by=["card1","addr1","D9"])
dff_s = dff.sort(by=["addr1","D9","card1"])
dff_s = dff.sort(by=["D9","card1","addr1"])
te = time()
s["sorting"] = te-ts

In [23]:
ts = time()
dff["card2"].value_counts()
te = time()
s["value_counts"] = te-ts

In [24]:
pd.DataFrame(stats)

,pandas,dask,dask_indexed,vaex
load_transactions,20.143912,0.090847,10.722161,16.696253
load_identity,0.501993,0.017655,0.582975,0.749975
merge,2.207136,0.075063,0.048098,0.120835
aggregation,0.062498,16.052712,14.059959,0.455176
sorting,1.258234,43.264110,40.348259,1.124968
value_counts,0.002657,NaN,NaN,0.021019


In [25]:
clean()

# PySpark
Java must be installed. (e.g. Ubuntu 18 - https://www.digitalocean.com/community/tutorials/how-to-install-java-with-apt-on-ubuntu-18-04)

In [26]:
from pyspark import SparkContext
sc = SparkContext()
sc.version
sc.stop()

In [27]:
# Create my_spark
my_spark = SparkSession.builder \
    .master("local") \
    .appName("Pandas Alternative") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [28]:
tool = "spark"
stats[tool] = {}
s = stats[tool]


ts = time()
df = my_spark.read.csv(paths[0],inferSchema = True,header= True) 
te = time()
s["load_transactions"] = te-ts

ts = time()
df2 = my_spark.read.csv(paths[1],inferSchema = True,header= True) 
te = time()
s["load_identity"] = te-ts

In [29]:

ts = time()
dff = df.join(df2, "TransactionID")
te = time()
s["merge"] = te-ts

In [30]:
# the difference is that we call collect method, which runs all the computations at this point
#ts = time()
#grp = dff.groupby([dff["isFraud"],dff["ProductCD"],dff["card4"],dff["card6"],dff["id_15"],dff["id_31"]]) \
#        .agg(avg("TransactionAmt"), sum("TransactionAmt"))\
#        .collect()
#te = time()
#s["aggregation"] = te-ts
#s["all"] = te-tss

In [31]:
# the difference is that we call collect method, which runs all the computations at this point
ts = time()
grp = dff.groupby(["isFraud","ProductCD","card4","card6","id_15","id_31"]) \
        .agg(avg("TransactionAmt"), sum("TransactionAmt"))\
        .collect()
te = time()
s["aggregation"] = te-ts


In [32]:
ts = time()
dff.orderBy("card1","addr1","D9").collect()
# alternatively
# dff.sort("card1","addr1","D9").collect()
te = time()
s["sorting"] = te-ts

In [33]:
ts = time()
dff.select("card2").distinct().collect()
te = time()
s["value_counts"] = te-ts

In [34]:
stats_df = pd.DataFrame(stats)
stats_df.loc['Total'] = stats_df.sum()
stats_df

,pandas,dask,dask_indexed,vaex,spark
load_transactions,20.143912,0.090847,10.722161,16.696253,32.811776
load_identity,0.501993,0.017655,0.582975,0.749975,1.250784
merge,2.207136,0.075063,0.048098,0.120835,0.302898
aggregation,0.062498,16.052712,14.059959,0.455176,14.660436
sorting,1.258234,43.264110,40.348259,1.124968,104.669759
value_counts,0.002657,NaN,NaN,0.021019,9.147272
Total,24.176429,59.500387,65.761452,19.168226,162.842925


# Modin

In [35]:
mpd.__version__

'0.8.2'

In [36]:
tool = "modin"
stats[tool] = {}
s = stats[tool]


ts = time()
df = mpd.read_csv(paths[0])
te = time()
s["load_transactions"] = te-ts

ts = time()
df2 = mpd.read_csv(paths[1])
te = time()
s["load_identity"] = te-ts

ts = time()
dff = df.merge(df2, on="TransactionID")
te = time()
s["merge"] = te-ts

# modin defaults to pandas for multiple column aggregation and then fails on KeyError, though the key is available
ts = time()
try:
    grp = dff.groupby(["isFraud","ProductCD","card4","card6","id_15","id_31"])["TransactionAmt"].agg(["mean","sum"])
except Exception as e:
    print(e)
te = time()
s["aggregation"] = te-ts


To request implementation, send an email to feature_requests@modin.org.


'isFraud'


In [37]:
pd.DataFrame(stats)

,pandas,dask,dask_indexed,vaex,spark,modin
load_transactions,20.143912,0.090847,10.722161,16.696253,32.811776,14.306013
load_identity,0.501993,0.017655,0.582975,0.749975,1.250784,0.777875
merge,2.207136,0.075063,0.048098,0.120835,0.302898,9.725335
aggregation,0.062498,16.052712,14.059959,0.455176,14.660436,0.341296
sorting,1.258234,43.264110,40.348259,1.124968,104.669759,NaN
value_counts,0.002657,NaN,NaN,0.021019,9.147272,NaN


In [38]:
clean()

In [39]:
pd.DataFrame(stats).to_csv("statistics.csv")

# Performance test

In [40]:
import psutil
import numpy as np

In [41]:
# use psutil to check for system resources
psutil.cpu_percent(), psutil.virtual_memory().percent

(62.9, 82.8)

In [42]:
def system_resources(n, pause, cpu_threshold = 0.5, mem_threshold = 0.5):
    
    cpu_m = []
    mem_m = []
    cpu_treshold_breached = False
    mem_treshold_breached = False
    
    for i in range(n):
        cpu_m.append(psutil.cpu_percent())
        mem_m.append(psutil.virtual_memory().percent)
        sleep(pause)
    cpu_m = np.mean(cpu_m)
    mem_m = np.mean(mem_m)
    
    if cpu_m / 100 > cpu_threshold:
        cpu_treshold_breached = True
        
    if mem_m / 100 > mem_threshold:
        mem_treshold_breached = True
    
    return {"cpu": cpu_m, "memory": mem_m, "cpu_threshold": cpu_treshold_breached, "mem_threshold": mem_treshold_breached }

In [43]:
system_resources(3, 1)

{'cpu': 25.599999999999998,
 'memory': 82.76666666666667,
 'cpu_threshold': False,
 'mem_threshold': True}

In [44]:
class Events:
    
    def __init__(self, path):
        self.file = open(path, 'a', encoding='utf-8')
        
    def log(self, time, tool, operation, duration):
        print("|".join([strftime('%Y-%m-%d %H:%M:%S', localtime(te)),tool,operation,str(duration)])+"\n")
        print(self.file)
        self.file.write("|".join([strftime('%Y-%m-%d %H:%M:%S', localtime(te)),tool,operation,str(duration)])+"\n")
        
    def close(self):
        self.file.close()

In [45]:
logger = Events("example.log")

In [46]:
ts = time()
df = pd.read_csv(paths[1])
te = time()
logger.log(te, "pandas", "load",  te-ts)

2021-01-25 22:38:38|pandas|load|0.552954912185669

<_io.TextIOWrapper name='example.log' mode='a' encoding='utf-8'>


In [47]:
logger.file.write("|".join([strftime('%Y-%m-%d %H:%M:%S', localtime(te)),"pandas","load",str(ts-te)])+"\n")

51

In [48]:
te = time()


In [49]:
import logging
logging.basicConfig(filename='/Notebooks/Medium/DataFrames/lexample.log', encoding='utf-8', level=logging.DEBUG)

In [50]:
logging.info('This is an info message')

INFO:MainThread:root:This is an info message
